In [1]:
from sqlalchemy import create_engine, text, inspect
import pandas as pd
db_path = r'C:\Users\vivia\co-pilot-v1\Notebooks\bulk_upload.db'

engine = create_engine(f'sqlite:///{db_path}')
# Create an inspector
inspector = inspect(engine)

# Get the list of table names
tables = inspector.get_table_names()

# Print the structure of each table
for table in tables:
    print(f"\nTable: {table}")
    columns = inspector.get_columns(table)
    for column in columns:
        print(f"  Column: {column['name']} - {column['type']}")

# Additionally, print foreign key information
for table in tables:
    print(f"\nForeign Keys for Table: {table}")
    foreign_keys = inspector.get_foreign_keys(table)
    for fk in foreign_keys:
        print(f"  Foreign Key: {fk['constrained_columns']} -> {fk['referred_table']}.{fk['referred_columns']}")

with engine.connect() as connection:
    query = text("""SELECT p.part_number, m.model_name, p.price, p.quantity, p.make, p.description, p.months_no_sale, p.value, y.year, p.uploaded
                    FROM model_year my
                    JOIN models m ON my.model_id=m.id
                    JOIN parts p ON my.part_number=p.part_number
                    JOIN years y ON my.year_id=y.id
                 """)
    df = connection.execute(query)
df = pd.DataFrame(df.fetchall())
df


Table: model_year
  Column: id - INTEGER
  Column: part_number - INTEGER
  Column: model_id - INTEGER
  Column: year_id - INTEGER

Table: models
  Column: id - INTEGER
  Column: part_number - VARCHAR
  Column: model_name - VARCHAR

Table: parts
  Column: part_number - VARCHAR
  Column: description - VARCHAR
  Column: quantity - INTEGER
  Column: value - FLOAT
  Column: price - FLOAT
  Column: make - VARCHAR
  Column: months_no_sale - INTEGER
  Column: uploaded - BOOLEAN

Table: years
  Column: id - INTEGER
  Column: part_number - VARCHAR
  Column: year - INTEGER

Foreign Keys for Table: model_year
  Foreign Key: ['part_number'] -> parts.['part_number']
  Foreign Key: ['model_id'] -> models.['id']
  Foreign Key: ['year_id'] -> years.['id']

Foreign Keys for Table: models
  Foreign Key: ['part_number'] -> parts.['part_number']

Foreign Keys for Table: parts

Foreign Keys for Table: years
  Foreign Key: ['part_number'] -> parts.['part_number']


,part_number,model_name,price,quantity,make,description,months_no_sale,value,year,uploaded
0,OPTEON1234YF,Fit,6.48,1114,Honda,FREON PER OZ,0,7218.72,2018,0
1,OPTEON1234YF,Fit,6.48,1114,Honda,FREON PER OZ,0,7218.72,2019,0
2,OPTEON1234YF,Fit,6.48,1114,Honda,FREON PER OZ,0,7218.72,2020,0
3,OPTEON1234YF,Ridgeline,6.48,1114,Honda,FREON PER OZ,0,7218.72,2018,0
4,OPTEON1234YF,Ridgeline,6.48,1114,Honda,FREON PER OZ,0,7218.72,2019,0
...,...,...,...,...,...,...,...,...,...,...
26849,TUCHY-R54-5MA1,Accord,181.29,1,Honda,TU PEN YR545M,16,10.35,2018,0
26850,TUCHY-R54-5MA1,Accord,181.29,1,Honda,TU PEN YR545M,16,10.35,2017,0
26851,TUCHY-R54-5MA1,Accord,181.29,1,Honda,TU PEN YR545M,16,10.35,2019,0
26852,TUCHY-R54-5MA1,Accord,181.29,1,Honda,TU PEN YR545M,16,10.35,2021,0
